#  Generating new quantities of interest given a existing model, data, and sample.


A Stan program is structured into named program blocks, all of which are optional, but which must occur in the following order:
  + functions
  + data
  + transformed data
  + parameters
  + transformed parameters
  + model
  + generated quantities
  
The [generated quantities](https://mc-stan.org/docs/2_19/reference-manual/program-block-generated-quantities.html) block computes quantities of interest based on the data, transformed data, parameters, and transformed parameters.  It allows us to:

  + generate simulated data for model testing by forward sampling
  + generate predictions for new data
  + calculate posterior event probabilities, including multiple comparisons, sign tests, etc.
  + calculating posterior expectations
  + transform parameters for reporting
  + apply full Bayesian decision theory
  + calculate log likelihoods, deviances, etc. for model comparison

If we have already obtained a sample from a model conditioned on a dataset and are satisfied with the model and the fit of the sample, but would like to compute new quantities of interest, the `run_generated_quantities` method allows us to do so without the expense of re-running the sampler.  Instead it takes the existing sample, and for each draw in the drawset, runs the generated quantities block of the program using the per-draw parameter estimates to compute the quantities of interest.



## Example:  add posterior predictive checks to `bernoulli.stan`


In this example we use the CmdStan example model [bernoulli.stan](https://github.com/stan-dev/cmdstanpy/blob/master/test/data/bernoulli.stan)
and data file [bernoulli.data.json](https://github.com/stan-dev/cmdstanpy/blob/master/test/data/bernoulli.data.json) as our existing model and data.

We instantiate the model `bernoulli`,
as in the "Hello World" section
of the CmdStanPy [tutorial](../../cmdstanpy_tutorial.ipynb) notebook.

In [ ]:
import os
import os.path
from cmdstanpy import Model, StanFit, cmdstan_path

bernoulli_dir = os.path.join(cmdstan_path(), 'examples', 'bernoulli')
bernoulli_path = os.path.join(bernoulli_dir, 'bernoulli.stan')

# instantiate bernoulli model, compile Stan program
bernoulli_model = Model(stan_file=bernoulli_path)
bernoulli_model.compile()
print(bernoulli_model.code())

We create program [bernoulli_ppc.stan](https://github.com/stan-dev/cmdstanpy/blob/master/test/data/bernoulli_ppc.stan)
by adding a `generated quantities` block which generates a new data vector `y_rep` using the current estimate of theta.

In [ ]:
bernoulli_ppc_model = Model(stan_file='bernoulli_ppc.stan')
bernoulli_ppc_model.compile()
print(bernoulli_ppc_model.code())

As in the "Hello World" tutorial, we produce a sample from the posterior of the model conditioned on the data:

In [ ]:
# fit the model to the data
bern_data = os.path.join(bernoulli_dir, 'bernoulli.data.json')
bern_fit = bernoulli_model.sample(data=bern_data)

The arguments to the `run_generated_quantities` method are:
 + the data used to fit the model  (`bern_data`)
 + the list of the resulting stan csv files (`bern_fit.csv_files`)

In [ ]:
new_quantities = bernoulli_ppc_model.run_generated_quantities(data=bern_data, csv_files=bern_fit.csv_files)